# test qwen2

In [1]:
# 增加norm和rope的一致性
import os
import sys
import torch
import numpy  as np
from deepspeed.profiling.flops_profiler import get_model_profile  # type: ignore
current_dir = os.getcwd()
print(current_dir)
sys.path.append(os.path.join(current_dir, "steel_modify_from_qwen_1_5"))
from transformers import AutoConfig
from steel_modify_from_qwen_1_5.modeling_steel import Qwen2ForCausalLM
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)
config.use_cuda_rmsnorm = True
model = Qwen2ForCausalLM(config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
memory_allocated = torch.cuda.memory_allocated(device)
print("模型在显存中的占用大小：", memory_allocated, "字节")
for name, param in model.named_parameters():
    print(f"参数名称: {name}，数据类型: {param.dtype}")
    break
# print(model)

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-10 16:12:06,108] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible
/home/calfa100/gqs/Steel-LLM/pretrain_modify_from_TinyLlama/model


ModuleNotFoundError: No module named 'qwen_15_1_8B_chat'

In [4]:

seed = 666
torch.manual_seed(seed)  # 设置随机种子
torch.cuda.manual_seed(seed)  # 设置CUDA的随机种子
torch.cuda.manual_seed_all(seed)  # 如果使用多个GPU，设置所有GPU的随机种子
np.random.seed(seed)
fake_input = torch.randint(1, 10000, (1, 256),dtype=torch.long, device=model.device)
input_dict = {"input_ids": fake_input, "labels": fake_input.clone()}
model = model.to(torch.float32)
flops, macs, params = get_model_profile(model, kwargs=input_dict, print_profile=True, detailed=True)
print("FLOPs:", flops)
print("MACs:", macs)
print("Params:", params)
output = model(fake_input)
print(output.logits[:,:,:10])
print("*"*20)
#===========================================================
model = model.to(torch.bfloat16)
flops, macs, params = get_model_profile(model, kwargs=input_dict, print_profile=True, detailed=True)
print("FLOPs:", flops)
print("MACs:", macs)
print("Params:", params)
output = model(fake_input)
print(output.logits[:,:,:10])

[2024-05-10 02:10:56,400] [INFO] [profiler.py:1218:get_model_profile] Flops profiler warming-up...
[2024-05-10 02:10:56,420] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 1:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         1.84 B  
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       422.72 GMACs
fwd flops per GPU:                                  

# 测试 rms+rope一致性

In [3]:
from flash_attn.layers.rotary import apply_rotary_emb_func as __apply_rotary_emb_func
from flash_attn.ops.rms_norm import rms_norm as __rms_norm
from torch import nn
import torch
class Qwen2RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        Qwen2RMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

In [12]:
eps = 1e-6
dim = 16
input  = torch.randn([1,dim],device="cuda:0",dtype=torch.bfloat16)
torch_rmsnorm = Qwen2RMSNorm(dim, eps=eps).to("cuda:0")
print(torch_rmsnorm.weight.dtype)
output1 = torch_rmsnorm(input)
#=
output2 = __rms_norm(input, torch_rmsnorm.weight, torch_rmsnorm.variance_epsilon)
print(output1)
print(output2)

torch.float32
tensor([[ 0.3965, -0.6992, -1.8359, -0.8242, -1.3672,  1.4688,  0.1836,  1.0703,
         -1.0859, -1.9062, -0.2969, -0.3633, -0.9648, -0.0815,  0.1748, -0.3105]],
       device='cuda:0', grad_fn=<MulBackward0>)
tensor([[ 0.3965, -0.6992, -1.8359, -0.8242, -1.3672,  1.4688,  0.1836,  1.0703,
         -1.0859, -1.9062, -0.2969, -0.3633, -0.9648, -0.0815,  0.1748, -0.3105]],
       device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<DropoutAddLayerNormFnBackward>)


In [ ]:
# test recurrentgemma
from transformers import AutoConfig
from recurrentgemma.modeling_recurrent_gemma import RecurrentGemmaForCausalLM
config = AutoConfig.from_pretrained("./recurrentgemma",trust_remote_code=True)
model = RecurrentGemmaForCausalLM(config)
print(model)